## Checklist

### 진행예정
 1) 추가필요 열: 날짜 \
 2) 타 플랫폼과 합쳤을 때 형식 통일하도록 data type과 Nan 처리방법 등 정의 \
 3) 메리어트 계열사 제외처리 필요
 

### 완료
 1) page바꿔가면서 했을 때 상품들 연속성 CHK (limit=50에 따른 상품 누락 여부 확인) -> 이상없음. \
 2) page=1, 2 이상 구분할 때 [4], [1]이 아니라 '전체'를 가져오게 해야겠다. 시간 저녁되니까 숫자로 안된다. -> 해결 \
 3) today 기준시 기준으로, 한국보다 9시간 느림 -> 한국시로 맞추기 -> 해결
 4) 리뷰 추가 필요 -> 해결
 5) headers에서 app-version 나중에 업데이트 필요할 수도 있음 -> 업데이트 이전의 app-version 넣어도 이상없음 -> 해결
### 추후 체크 디버깅 예상
 1) X

In [10]:
import pandas as pd
import numpy as np

In [29]:
# 1. 프로젝트 생성

In [1]:
# !rm -rf dailyhotel
!scrapy startproject dailyhotel

New Scrapy project 'dailyhotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/03_ml/dailyhotel

You can start your first spider with:
    cd dailyhotel
    scrapy genspider example example.com


In [2]:
!tree dailyhotel

dailyhotel
├── dailyhotel
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

2 directories, 7 files


In [28]:
# 2. xpath 찾기: headers 설정

### hotel_key 가져오는 모듈 생성

In [10]:
# hotel key 얻는 모듈 생성 -> dailyhotel.py
def get_hotel_keys():
    import scrapy, requests
    from scrapy.http import TextResponse
    import datetime
    
    
    KST = datetime.timezone(datetime.timedelta(hours=9))
    today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
    start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
    end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
    staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
                datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
    persons, staydays = 1, 1
    
    
    # 호텔리스트 가져올 parent_url 생성
    page = 1
    hotelidx = []
    while True:
        parent_url = f"https://www.dailyhotel.com/newdelhi/goodnight/api/v4/stays/sales?dateCheckIn={start_date}&regionIdx=30070&shortCutType=hotel_resort&persons={persons}&stays={staydays}&details=true&regionStayType=hotel_resort&stayType=hotel,resort&page={page}&limit=50"
        headers = {'app-version': '2.2.14'}
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        list_all = response['data']['staySaleSections']

        if len(list_all) > 0:
            for each in list_all:
                if each['sectionViewType'] == 'NORMAL_섹션뷰타입':
                    lists = each['staySales']
                    print('page=', str(page))
                    page += 1
        else:
            print('All registered hotels have been searched.') # 이거는 확인용
            break


        # parent_url에서 각 호텔들의 Index number 가져오기
        for each in lists:
            hotelidx.append(each['hotelIdx'])
    
    return hotelidx

### 가져온 hotel_keys로 xpath찾기

In [22]:
# 일단 hotel_keys 받고 url 만들어서 yield시킬 예정
# 테스트
idx = hotelidx[0]
staydays = 1
start_date = "2021-03-10"


hotel_url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/{idx}?stays={staydays}&dateCheckIn={start_date}&regionStayType=all'
headers = {
    'app-version': '2.2.13'
}
req = requests.get(hotel_url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
resp_content = response.json()


#name
name = resp_content['data']['name'] 
#level
level = resp_content['data']['stayGradeName'] 
#score
try:
    score = round(resp_content['data']['rating']['values']/100*5, 1) 
except:
    score = '점수없음'
# location
location = resp_content['data']['address']
# roomtype
roomtype = []
for each in resp_content['data']['rooms']:
    roomtype.append(each['roomName'])
#price
price = []
for each in resp_content['data']['rooms']:
    price.append(each['amount']['discountAverage'])
#review
review = resp_content['data']['statistic']['reviewScoreTotalCount']
#link
hotel_desc_url = f"https://www.dailyhotel.com/stays/{idx}?dateCheckIn={start_date}&stays={staydays}" # 크롤링이랑 예약링크 다름 -> request url과 다르도록!

# print(name)
# print(level)
# print(score)
# print(location)
# print(roomtype)
# print(price)
# print(review)
# print(hotel_desc_url)

1491

In [12]:
# 3. item.py

In [3]:
%%writefile dailyhotel/dailyhotel/items.py
import scrapy


class DailyhotelItem(scrapy.Item):
    platform = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review = scrapy.Field()
    location = scrapy.Field()
    roomtype = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting dailyhotel/dailyhotel/items.py


# 4. spider.py

In [4]:
%%writefile dailyhotel/dailyhotel/spiders/spider.py
import scrapy, requests
import json
import datetime
from DailyhotelKeys import get_hotel_keys
from dailyhotel.items import DailyhotelItem


class DailyhotelSpider(scrapy.Spider):
    name = "Dailyhotel"
    allow_domain=["dailyhotel.com"]
    KST = datetime.timezone(datetime.timedelta(hours=9))
    today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
    start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
    end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
    staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
                datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
    persons, staydays = 2, 1
    start_urls= ["https://www.dailyhotel.com/"]
    
    
    def parse(self, response):
        hotel_idx = get_hotel_keys()
        KST = datetime.timezone(datetime.timedelta(hours=9))
        today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
        start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
        links = []
        start_urls = "https://www.dailyhotel.com/"
        headers = {'app-version': '2.2.14',
                   'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36',
                  }
        for idx in hotel_idx:
            hotel_url = start_urls + f'newdelhi/goodnight/api/v9/hotel/{idx}?stays=1&dateCheckIn={start_date}&regionStayType=all'
            links.append(hotel_url)   
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
    
    
    def parse_content(self, response):
        KST = datetime.timezone(datetime.timedelta(hours=9))
        today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
        start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘  
        resp_content = response.json()
        staydays=1        
        
        
        #name
        name = resp_content['data']['name'] 
        #level
        level = resp_content['data']['stayGradeName'] 
        #score
        try:
            score = round(resp_content['data']['rating']['values']/100*5, 1) 
        except:
            score = '점수없음'
        # location
        location = resp_content['data']['address']
        # roomtype
        roomtypes = []
        for each in resp_content['data']['rooms']:
            roomtypes.append(each['roomName'])
        #price
        prices = []
        for each in resp_content['data']['rooms']:
            prices.append(each['amount']['discountAverage'])
        #review
        review = resp_content['data']['statistic']['reviewScoreTotalCount']
        #link
        hotel_id = resp_content['data']['idx']
        hotel_desc_url = f"https://www.dailyhotel.com/stays/{hotel_id}?dateCheckIn={start_date}&stays={staydays}" # 크롤링이랑 예약링크 다름 -> request url과 다르도록!
        #platform
        platform = "데일리호텔"

        
        item = DailyhotelItem()
        for i in range(len(roomtypes)):            
            item['platform'] = platform
            item['name'] = name
            item['level'] = level
            item['score'] = score
            item['review'] = review
            item['location'] = location
            item['roomtype'] = roomtypes[i]
            item['price'] = prices[i]
            item['link'] = hotel_desc_url     
            yield item

Writing dailyhotel/dailyhotel/spiders/spider.py


In [30]:
# 5. settings.py: robots.txt 여부 파악

In [31]:
# 6. 프로젝트 실행

In [5]:
%pwd

'/home/ubuntu/python3/notebook/03_ml'

In [6]:
%%writefile run.sh
cd dailyhotel
rm dailyhotel.csv
scrapy crawl Dailyhotel -o dailyhotel.csv 

Overwriting run.sh


In [7]:
!/bin/bash run.sh

rm: cannot remove 'dailyhotel.csv': No such file or directory
2021-03-11 18:02:56 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: dailyhotel)
2021-03-11 18:02:56 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jan 15 2021, 05:57:06) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-11 18:02:56 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-11 18:02:56 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'dailyhotel',
 'NEWSPIDER_MODULE': 'dailyhotel.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['dailyhotel.spiders']}
2021-03-11 18:02:56 [scrapy.extensions.telnet] INFO: Telnet Password: 3dd963ce39eb0495
2021-03-11 18:02:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions

2021-03-11 18:03:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/36127?stays=1&dateCheckIn=2021-03-11&regionStayType=all> (referer: https://www.dailyhotel.com/)
2021-03-11 18:03:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/27914?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/27914?dateCheckIn=2021-03-11&stays=1',
 'location': '서울 용산구 청파로20길 95',
 'name': '노보텔 스위트 앰배서더 서울 용산',
 'platform': '데일리호텔',
 'price': 178200,
 'review': 2427,
 'roomtype': '[HOTDEAL!] 주니어 스위트 ',
 'score': 4.7}
2021-03-11 18:03:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/27914?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/27914?dateCheckIn=2021-03-11&stays=1',
 'location': '서울 용산구 청파로20길 95',
 'name': 

2021-03-11 18:03:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/36176?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/36176?dateCheckIn=2021-03-11&stays=1',
 'location': '서울특별시 종로구 인사동길 49',
 'name': '나인트리 프리미어 호텔 인사동',
 'platform': '데일리호텔',
 'price': 66000,
 'review': 1484,
 'roomtype': '[DAILY단독ㅣ연박전용 Green Earth by Nine Tree] Standard Twin ',
 'score': 4.8}
2021-03-11 18:03:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/36176?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/36176?dateCheckIn=2021-03-11&stays=1',
 'location': '서울특별시 종로구 인사동길 49',
 'name': '나인트리 프리미어 호텔 인사동',
 'platform': '데일리호텔',
 'price': 66000,
 'review': 1484,
 'roomtype': '[DAILY단독ㅣ연박전용 Green Earth by Nine Tree] Standard Double ',
 'score': 4.8}
2021-03-11 18:03:02 [scr

2021-03-11 18:03:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/1439?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/1439?dateCheckIn=2021-03-11&stays=1',
 'location': '서울 구로구 경인로 662',
 'name': '쉐라톤 서울 디큐브시티 호텔',
 'platform': '데일리호텔',
 'price': 152900,
 'review': 2909,
 'roomtype': '[와인 PKG] 디럭스 ',
 'score': 4.5}
2021-03-11 18:03:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/1439?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/1439?dateCheckIn=2021-03-11&stays=1',
 'location': '서울 구로구 경인로 662',
 'name': '쉐라톤 서울 디큐브시티 호텔',
 'platform': '데일리호텔',
 'price': 165000,
 'review': 2909,
 'roomtype': '[Stay & Relax 30] 디럭스 ',
 'score': 4.5}
2021-03-11 18:03:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodni

2021-03-11 18:03:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/27917?stays=1&dateCheckIn=2021-03-11&regionStayType=all> (referer: https://www.dailyhotel.com/)
2021-03-11 18:03:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/32074?stays=1&dateCheckIn=2021-03-11&regionStayType=all> (referer: https://www.dailyhotel.com/)
2021-03-11 18:03:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/1419?stays=1&dateCheckIn=2021-03-11&regionStayType=all> (referer: https://www.dailyhotel.com/)
2021-03-11 18:03:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/367?stays=1&dateCheckIn=2021-03-11&regionStayType=all> (referer: https://www.dailyhotel.com/)
2021-03-11 18:03:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/ap

...

2021-03-11 18:03:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/31121?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/31121?dateCheckIn=2021-03-11&stays=1',
 'location': '서울시 마포구 양화로 188 애경타워 숙박시설동',
 'name': '홀리데이인 익스프레스 서울 홍대',
 'platform': '데일리호텔',
 'price': 68800,
 'review': 1552,
 'roomtype': '유료주차ㅣ[☆특가데일리ㅣ2인조식포함] Standard Queen ',
 'score': 4.8}
2021-03-11 18:03:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/31121?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/31121?dateCheckIn=2021-03-11&stays=1',
 'location': '서울시 마포구 양화로 188 애경타워 숙박시설동',
 'name': '홀리데이인 익스프레스 서울 홍대',
 'platform': '데일리호텔',
 'price': 71800,
 'review': 1552,
 'roomtype': '[특가데일리☆늦첵 특가-20시이후체크인] Superior Twin(유료주차) ',
 'score': 4.8}
2021-03-11 18:03:09 [scrapy.core.sc

2021-03-11 18:03:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/1415?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '1급',
 'link': 'https://www.dailyhotel.com/stays/1415?dateCheckIn=2021-03-11&stays=1',
 'location': '서울특별시 서대문구 충정로 76',
 'name': '신라스테이 서대문',
 'platform': '데일리호텔',
 'price': 80000,
 'review': 3637,
 'roomtype': '[DAILY 단독ㅣ레이트체크아웃14시] 프리미어 디럭스 더블 ',
 'score': 4.5}
2021-03-11 18:03:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/1415?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '1급',
 'link': 'https://www.dailyhotel.com/stays/1415?dateCheckIn=2021-03-11&stays=1',
 'location': '서울특별시 서대문구 충정로 76',
 'name': '신라스테이 서대문',
 'platform': '데일리호텔',
 'price': 80000,
 'review': 3637,
 'roomtype': '[DAILY 단독ㅣ레이트체크아웃 14시] 프리미어 디럭스 패밀리트윈 ',
 'score': 4.5}
2021-03-11 18:03:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dai

2021-03-11 18:03:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/35556?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/35556?dateCheckIn=2021-03-11&stays=1',
 'location': '서울특별시 중구 칠패로 27',
 'name': '라마다 호텔앤스위트 서울 남대문',
 'platform': '데일리호텔',
 'price': 82500,
 'review': 692,
 'roomtype': '디럭스 트윈 ',
 'score': 4.1}
2021-03-11 18:03:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/13382?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/13382?dateCheckIn=2021-03-11&stays=1',
 'location': '서울 강남구 테헤란로 606',
 'name': '파크 하얏트 서울',
 'platform': '데일리호텔',
 'price': 288800,
 'review': 755,
 'roomtype': '[단독★Good Evening PKG] 스탠다드 1킹→디럭스 1킹 UP ',
 'score': 4.7}
2021-03-11 18:03:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newd

2021-03-11 18:03:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/29362?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '1급',
 'link': 'https://www.dailyhotel.com/stays/29362?dateCheckIn=2021-03-11&stays=1',
 'location': '서울 중구 동호로 354',
 'name': '라마다 서울 동대문',
 'platform': '데일리호텔',
 'price': 85000,
 'review': 995,
 'roomtype': '[★30시간 STAY★] 슈페리어 ',
 'score': 4.5}
2021-03-11 18:03:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/29362?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '1급',
 'link': 'https://www.dailyhotel.com/stays/29362?dateCheckIn=2021-03-11&stays=1',
 'location': '서울 중구 동호로 354',
 'name': '라마다 서울 동대문',
 'platform': '데일리호텔',
 'price': 85000,
 'review': 995,
 'roomtype': '[★30시간 STAY★] 슈페리어 ',
 'score': 4.5}
2021-03-11 18:03:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/

2021-03-11 18:03:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/27910?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/27910?dateCheckIn=2021-03-11&stays=1',
 'location': '서울 용산구 청파로20길 95',
 'name': '그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산',
 'platform': '데일리호텔',
 'price': 193100,
 'review': 745,
 'roomtype': '[HOT DEAL!] 주니어 스위트 ',
 'score': 4.8}
2021-03-11 18:03:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/27910?stays=1&dateCheckIn=2021-03-11&regionStayType=all>
{'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/27910?dateCheckIn=2021-03-11&stays=1',
 'location': '서울 용산구 청파로20길 95',
 'name': '그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산',
 'platform': '데일리호텔',
 'price': 214500,
 'review': 745,
 'roomtype': '주니어 스위트 ',
 'score': 4.8}
2021-03-11 18:03:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.daily

In [8]:
# 실행시간 12초

In [11]:
data = pd.read_csv('dailyhotel/dailyhotel.csv', encoding='utf-8-sig')
data.shape

(2028, 9)

In [12]:
data.tail()

,level,link,location,name,platform,price,review,roomtype,score
2023,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,435100.0,745,[HOT DEAL!] 디럭스 스위트 2베드룸,4.8
2024,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,456500.0,745,디럭스 스위트 2베드룸,4.8
2025,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,496700.0,745,[HOT DEAL!ㅣ조식 2인 포함] 디럭스 스위트 2베드룸,4.8
2026,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,518100.0,745,[조식 PKG] 디럭스 스위트 2베드룸,4.8
2027,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,567100.0,745,[The Lounge l EFL 2인 포함PKG] 디럭스 스위트 2베드룸,4.8
